DATA643- Project 2
======
Prashanth Padebetttu, Parshu Rath, Adjare Windokun
----------
<b> Summer 2016 <br>
Instructor: Andrew Catlin

<b> Load 100K movie data as pandas dataframe:

In [25]:
# Movielens data with 100,000 records
from math import sqrt

In [2]:
#Load data into a pandas dataframe
import csv
#import os
import pandas as pd
import numpy as np
from scipy import spatial
url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/IS-607-Data-Acquisition-and-Management/IS643/Project_2/movielens.csv'
df = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')


In [3]:
df.head()

,userid,age,gender,occupation,zipcode,itemid,rating,timestamp,movieid,movietitle,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,196,49,M,writer,55105,242,3,881250949,242,Kolya (1996),...,0,0,0,0,0,0,0,0,0,0
1,186,39,F,executive,00000,302,3,891717742,302,L.A. Confidential (1997),...,0,1,0,0,1,0,0,1,0,0
2,22,25,M,writer,40206,377,1,878887116,377,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,0
3,244,28,M,technician,80525,51,2,880606923,51,Legends of the Fall (1994),...,0,0,0,0,0,1,0,0,1,1
4,166,47,M,educator,55113,346,1,886397596,346,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,0


<b>Load 100K movie data as dictionary:

In [4]:
# Get movie titles
#url = 'https://raw.githubusercontent.com/prcuny/DATA643/master/Project-2/'
movies={}
#for line in open(url+'u.item', 'r'):  #does not work yet
for line in open('u.item'):    
    (id,title)=line.split('|')[0:2]
    movies[id]=title


In [5]:
# Load user ratings
mdict={}
for line in open('u.data'):
   (user,movieid,rating,ts)=line.split('\t')
   mdict.setdefault(user,{})
   mdict[user][movies[movieid]]=float(rating)


Print first 2 users and their ratings

In [21]:
#user '20' and ratings
#mdict['20']
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
take(2, mdict.iteritems())

[('344',
  {'Absolute Power (1997)': 3.0,
   'Addicted to Love (1997)': 3.0,
   'Aladdin (1992)': 4.0,
   'Alien (1979)': 5.0,
   'Aliens (1986)': 5.0,
   'Amadeus (1984)': 5.0,
   "Antonia's Line (1995)": 4.0,
   'Apt Pupil (1998)': 5.0,
   'As Good As It Gets (1997)': 4.0,
   'Babe (1995)': 5.0,
   'Back to the Future (1985)': 4.0,
   'Beautiful Girls (1996)': 3.0,
   'Beautiful Thing (1996)': 3.0,
   'Beauty and the Beast (1991)': 5.0,
   'Bed of Roses (1996)': 3.0,
   'Before Sunrise (1995)': 4.0,
   'Being There (1979)': 5.0,
   'Big Bully (1996)': 1.0,
   'Big Night (1996)': 5.0,
   'Birdcage, The (1996)': 4.0,
   'Blade Runner (1982)': 5.0,
   'Bound (1996)': 4.0,
   'Braveheart (1995)': 3.0,
   'Brazil (1985)': 5.0,
   'Breakdown (1997)': 3.0,
   'Broken Arrow (1996)': 3.0,
   'Brothers McMullen, The (1995)': 3.0,
   'Cable Guy, The (1996)': 1.0,
   'Chasing Amy (1997)': 4.0,
   'City Hall (1996)': 3.0,
   'Close Shave, A (1995)': 5.0,
   'Clueless (1995)': 5.0,
   'Contact (19

### User-User Collaborative Filtering

<b> Pearson correlation

In [26]:
#Function that returns the Pearson correlation coefficient for person1 and person2 
def sim_pearson(dict,p1,p2):
 #Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 for item in dict[p1]:
  if item in dict[p2] and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])): 
   si[item] = 1

 #if they are no rating in common, return 0
 if len(si) == 0:
  return 0

 #sum calculations
 n = len(si)
            
 #sum of all preferences
 sum1 = sum([dict[p1][it] for it in si])
 sum2 = sum([dict[p2][it] for it in si])
 #Sum of the squares
 sum1Sq = sum([pow(dict[p1][it],2) for it in si])
 sum2Sq = sum([pow(dict[p2][it],2) for it in si])
 #Sum of the products
 pSum = sum([dict[p1][it] * dict[p2][it] for it in si])
 #Calculate r (Pearson score)
 num = pSum - (sum1 * sum2)/n
 den = sqrt((sum1Sq - pow(sum1,2)/n) * (sum2Sq - pow(sum2,2)/n))
 if den == 0:
  return 0
 r = num/den
 return r


In [43]:
# Test the Pearson-based similarity function
round(sim_pearson(mdict,'20','30'), 4)

-0.4229

<b> Recommendations by Pearson weighted average: 

In [38]:
#Function that returns 'n' recommendations for a person by using a weighted average of every other user's rankings

def getRecommendations(n, dict,person,similarity=sim_pearson):
 totals = {}
 simSums = {}

 for other in dict:
  #don't compare me to myself
  if other == person:
   continue
  sim = similarity(dict,person,other)
  #ignore scores of zero or lower
  if sim <= 0: 
   continue
  for item in dict[other]:
   #only score movies I haven't rated yet
   if (item not in dict[person]) or (dict[person][item] == 0) or (np.isnan(dict[person][item])) :
    #Similarity * score
    totals.setdefault(item,0)
    totals[item] += not(np.isnan(dict[other][item] * sim))
    #Sum of similarities
    simSums.setdefault(item,0)
    simSums[item] += sim

    #Create the normalized list
 rankings = [(total/simSums[item],item) for item,total in totals.items()]

 #Return the sorted list
 rankings.sort()
 rankings.reverse()
 return rankings[0:n]


<b> Get top 10 recommendations for user '20' using User-User CF

In [40]:
#Test getRecommendations function
getRecommendations(10, mdict,'20',similarity=sim_pearson)

[(236.52695406660396, 'Golden Earrings (1947)'),
 (88.44403280479239, 'Johns (1996)'),
 (88.44403280479239, 'Beans of Egypt, Maine, The (1994)'),
 (87.25594535617968, 'Paradise Road (1997)'),
 (55.054047052742746, 'Cosi (1996)'),
 (55.054047052742746, 'All Over Me (1997)'),
 (53.94433001475551, 'Favor, The (1994)'),
 (49.580099539706296,
  'Brother Minister: The Assassination of Malcolm X (1994)'),
 (37.8437564385548, 'You So Crazy (1994)'),
 (37.8437564385548, 'Walking Dead, The (1995)')]